In [1]:
#import the working library
import numpy as np
import pandas as pd
import os
from scipy.stats.mstats import winsorize
from scipy.stats import zscore
%matplotlib inline

In [2]:
#read tickers as specified in excel in addition to market 
all_tickers = pd.read_excel('mapping.xlsx')[['Ticker', 'Market', 'Include']]
#list all tickers
all_tickers = all_tickers[all_tickers['Include']]
all_tickers

,Ticker,Market,Include
0,ARGT,Emerging,True
1,EWA,Developed,True
2,EWO,Developed,True
3,EWK,Developed,True
4,EWZ,Emerging,True
5,EWC,Developed,True
6,ECH,Emerging,True
7,MCHI,Emerging,True
12,EWQ,Developed,True
13,EWG,Developed,True


In [3]:
factor_description = pd.read_excel('factor description.xlsx', index_col='Short Name')
factor_description

,Name,Category
Short Name,,
mom_21,Short-term Absolute Momentum (21-trading day),Technical
mom_63,Medium-term Absolute Momentum (63-trading day),Technical
mom_252,Long-term Absolute Momentum (252-trading day),Technical
mr_21,Percentage over Short-term Moving Average (21-...,Technical
mr_63,Percentage over Medium-term Moving Average (63...,Technical
mr_252,Percentage over Long-term Moving Average (252-...,Technical
div_yield,Dividend Yield,Fundamental
pb,Price/Book Ratio,Fundamental
pe,Price/Earning Ratio,Fundamental


In [4]:
#specified factor and date
start_date_train = '2016-01-01'
end_date_train = '2018-12-31'
start_date_test = '2019-01-01'
end_date_test = '2019-12-31'

# Technical Factors

In [5]:
factors = ['mom_252', 'mom_63', 'mom_21', 
           'mr_252', 'mr_63', 'mr_21']

In [6]:
all_etf_all_factor_df = None
for factor_to_analyze in factors:
    description_factor_to_analyze = factor_description.loc[factor_to_analyze, 'Name']
    print(description_factor_to_analyze)

    #list all etf in datafrome
    all_etf_combined_df = pd.DataFrame()

    #going throw each etf forward return 
    for file in os.listdir('forward return'):
        fr_df = pd.read_csv('forward return/'+file, index_col='Date')
        fr_df['Ticker'] = file[:-4]
        try:
            factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
            combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'Factor'}, axis=1)
            combined_df['Factor'] = combined_df['Factor'].fillna(method='ffill')
            all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
        except:
            continue
    
    all_etf_combined_df = all_etf_combined_df.join(all_tickers.set_index('Ticker'), on='Ticker').dropna()
    all_etf_combined_df = all_etf_combined_df.sort_values(['Date', 'Market', 'Ticker']).set_index(['Date', 'Market', 'Ticker'])[start_date_train:end_date_test]
    all_etf_combined_df['Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor'] = np.nan

    #combine all factors
    for date in all_etf_combined_df.index.get_level_values(0).unique():
        for market in ['Developed', 'Emerging']:
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            if n_etf >= 15:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = winsorize(all_etf_combined_df.loc[date].loc[market]['Factor'], limits=1/n_etf)
            elif n_etf > 0:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = all_etf_combined_df.loc[date].loc[market]['Factor']
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'] = zscore(all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'])
    all_etf_combined_df = all_etf_combined_df.rename({'Normalized Winsorized Factor': factor_to_analyze}, axis=1)
    if all_etf_all_factor_df is None:
        all_etf_all_factor_df = all_etf_combined_df[['f_rtn', factor_to_analyze]]
    else:
        all_etf_all_factor_df = all_etf_all_factor_df.join(all_etf_combined_df[[factor_to_analyze]], how='outer')

Long-term Absolute Momentum (252-trading day)
Medium-term Absolute Momentum (63-trading day)
Short-term Absolute Momentum (21-trading day)
Percentage over Long-term Moving Average (252-trading day)
Percentage over Medium-term Moving Average (63-trading day)
Percentage over Short-term Moving Average (21-trading day)


In [7]:
all_etf_all_factor_df.reset_index()

,Date,Market,Ticker,f_rtn,mom_252,mom_63,mom_21,mr_252,mr_63,mr_21
0,2016-01-04,Developed,ENZL,-0.035457,0.302985,1.561762,2.005201,1.970015,1.891595,2.293367
1,2016-01-04,Developed,EWA,-0.071979,-0.818541,1.183367,0.877727,-0.222320,1.475917,2.293367
2,2016-01-04,Developed,EWC,-0.013368,-1.876307,-1.625714,-1.385869,-1.443837,-1.532190,-0.567871
3,2016-01-04,Developed,EWD,-0.060830,-0.192694,-0.601391,-1.026347,-0.496283,-0.379806,-0.578059
4,2016-01-04,Developed,EWG,-0.055149,-0.039418,0.630081,-0.672669,-0.327268,-0.178206,-1.153282
...,...,...,...,...,...,...,...,...,...,...
35170,2019-12-30,Emerging,GREK,-0.038922,2.182578,0.248636,-0.403086,1.538438,0.172329,0.134355
35171,2019-12-30,Emerging,KSA,-0.021221,-0.476315,-1.026486,-0.254276,-1.018996,-0.205677,-0.982870
35172,2019-12-30,Emerging,MCHI,-0.042813,0.552846,1.214980,0.164630,0.599531,1.017835,0.578072
35173,2019-12-30,Emerging,THD,-0.082510,-0.423942,-1.501269,-1.660978,-0.933935,-1.494013,-1.033223


In [8]:
all_etf_all_factor_df.reset_index().to_csv('modeling data/technical.csv')

# Fundamental Factors

In [9]:
factors = ['div_yield', 'pe', 'pb', 'pcf']

In [10]:
all_etf_all_factor_df = None
for factor_to_analyze in factors:
    description_factor_to_analyze = factor_description.loc[factor_to_analyze, 'Name']
    print(description_factor_to_analyze)

    #list all etf in datafrome
    all_etf_combined_df = pd.DataFrame()

    #going throw each etf forward return 
    for file in os.listdir('forward return'):
        fr_df = pd.read_csv('forward return/'+file, index_col='Date')
        fr_df['Ticker'] = file[:-4]
        try:
            factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
            combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'Factor'}, axis=1)
            combined_df['Factor'] = combined_df['Factor'].fillna(method='ffill')
            all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
        except:
            continue
    
    all_etf_combined_df = all_etf_combined_df.join(all_tickers.set_index('Ticker'), on='Ticker').dropna()
    all_etf_combined_df = all_etf_combined_df.sort_values(['Date', 'Market', 'Ticker']).set_index(['Date', 'Market', 'Ticker'])[start_date_train:end_date_test]
    all_etf_combined_df['Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor'] = np.nan

    #combine all factors
    for date in all_etf_combined_df.index.get_level_values(0).unique():
        for market in ['Developed', 'Emerging']:
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            if n_etf >= 15:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = winsorize(all_etf_combined_df.loc[date].loc[market]['Factor'], limits=1/n_etf)
            elif n_etf > 0:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = all_etf_combined_df.loc[date].loc[market]['Factor']
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'] = zscore(all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'])
    all_etf_combined_df = all_etf_combined_df.rename({'Normalized Winsorized Factor': factor_to_analyze}, axis=1)
    if all_etf_all_factor_df is None:
        all_etf_all_factor_df = all_etf_combined_df[['f_rtn', factor_to_analyze]]
    else:
        all_etf_all_factor_df = all_etf_all_factor_df.join(all_etf_combined_df[[factor_to_analyze]], how='outer')

Dividend Yield
Price/Earning Ratio
Price/Book Ratio
Price/Cash Flow Ratio


In [11]:
all_etf_all_factor_df.reset_index()

,Date,Market,Ticker,f_rtn,div_yield,pe,pb,pcf
0,2016-01-04,Developed,ENZL,-0.035457,1.681060,-0.177125,0.284371,0.596758
1,2016-01-04,Developed,EWA,-0.071979,1.545555,-0.258636,0.208013,-0.613292
2,2016-01-04,Developed,EWG,-0.055149,-0.467115,-0.249682,-0.020659,-0.735939
3,2016-01-04,Developed,EWH,-0.098062,-0.155762,-1.123668,-1.086060,2.674202
4,2016-01-04,Developed,EWI,-0.141762,0.557779,2.912695,-1.279059,-0.660771
...,...,...,...,...,...,...,...,...
31653,2019-12-30,Emerging,GREK,-0.038922,-0.340118,1.667498,1.529241,2.102185
31654,2019-12-30,Emerging,KSA,-0.021221,1.559869,1.667498,0.594718,0.267220
31655,2019-12-30,Emerging,MCHI,-0.042813,-1.400002,-0.360158,0.211644,-0.377070
31656,2019-12-30,Emerging,THD,-0.082510,-0.196165,0.550064,0.596054,-0.300105


In [12]:
all_etf_all_factor_df.reset_index().to_csv('modeling data/fundamental.csv')

# Economic Factors

In [13]:
factors = ['gdp_growth', 'debt_gdp', 'reer']

In [14]:
all_etf_all_factor_df = None
for factor_to_analyze in factors:
    description_factor_to_analyze = factor_description.loc[factor_to_analyze, 'Name']
    print(description_factor_to_analyze)

    #list all etf in datafrome
    all_etf_combined_df = pd.DataFrame()

    #going throw each etf forward return 
    for file in os.listdir('forward return'):
        fr_df = pd.read_csv('forward return/'+file, index_col='Date')
        fr_df['Ticker'] = file[:-4]
        try:
            factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
            combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'Factor'}, axis=1)
            combined_df['Factor'] = combined_df['Factor'].fillna(method='ffill')
            all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
        except:
            continue
    
    all_etf_combined_df = all_etf_combined_df.join(all_tickers.set_index('Ticker'), on='Ticker').dropna()
    all_etf_combined_df = all_etf_combined_df.sort_values(['Date', 'Market', 'Ticker']).set_index(['Date', 'Market', 'Ticker'])[start_date_train:end_date_test]
    all_etf_combined_df['Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor'] = np.nan

    #combine all factors
    for date in all_etf_combined_df.index.get_level_values(0).unique():
        for market in ['Developed', 'Emerging']:
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            if n_etf >= 15:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = winsorize(all_etf_combined_df.loc[date].loc[market]['Factor'], limits=1/n_etf)
            elif n_etf > 0:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = all_etf_combined_df.loc[date].loc[market]['Factor']
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'] = zscore(all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'])
    all_etf_combined_df = all_etf_combined_df.rename({'Normalized Winsorized Factor': factor_to_analyze}, axis=1)
    if all_etf_all_factor_df is None:
        all_etf_all_factor_df = all_etf_combined_df[['f_rtn', factor_to_analyze]]
    else:
        all_etf_all_factor_df = all_etf_all_factor_df.join(all_etf_combined_df[[factor_to_analyze]], how='outer')

QoQ GDP Growth
National Debt/GDP Ratio
REER


In [15]:
all_etf_all_factor_df.reset_index()

,Date,Market,Ticker,f_rtn,gdp_growth,debt_gdp,reer
0,2016-01-04,Developed,ENZL,-0.035457,1.820232,-1.411809,0.357234
1,2016-01-04,Developed,EWA,-0.071979,0.603763,-1.119388,-0.911989
2,2016-01-04,Developed,EWC,-0.013368,-1.144913,0.609708,-1.827849
3,2016-01-04,Developed,EWD,-0.060830,1.820232,-1.151173,-0.266560
4,2016-01-04,Developed,EWG,-0.055149,-0.384619,-0.261197,-0.458566
...,...,...,...,...,...,...,...
35170,2019-12-30,Emerging,GREK,-0.038922,0.184934,1.736160,-0.263181
35171,2019-12-30,Emerging,KSA,-0.021221,-0.346825,-1.439492,1.250170
35172,2019-12-30,Emerging,MCHI,-0.042813,1.780211,-1.420993,1.446190
35173,2019-12-30,Emerging,THD,-0.082510,-0.027770,0.472066,1.446190


In [16]:
all_etf_all_factor_df.reset_index().to_csv('modeling data/economic.csv')

# Sentimental and Alternative Factors

In [17]:
factors = ['pc_ratio', 'short_int_ratio']

In [18]:
all_etf_all_factor_df = None
for factor_to_analyze in factors:
    description_factor_to_analyze = factor_description.loc[factor_to_analyze, 'Name']
    print(description_factor_to_analyze)

    #list all etf in datafrome
    all_etf_combined_df = pd.DataFrame()

    #going throw each etf forward return 
    for file in os.listdir('forward return'):
        fr_df = pd.read_csv('forward return/'+file, index_col='Date')
        fr_df['Ticker'] = file[:-4]
        try:
            factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
            combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'Factor'}, axis=1)
            combined_df['Factor'] = combined_df['Factor'].fillna(method='ffill')
            all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
        except:
            continue
    
    all_etf_combined_df = all_etf_combined_df.join(all_tickers.set_index('Ticker'), on='Ticker').dropna()
    all_etf_combined_df = all_etf_combined_df.sort_values(['Date', 'Market', 'Ticker']).set_index(['Date', 'Market', 'Ticker'])[start_date_train:end_date_test]
    all_etf_combined_df['Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor'] = np.nan

    #combine all factors
    for date in all_etf_combined_df.index.get_level_values(0).unique():
        for market in ['Developed', 'Emerging']:
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            if n_etf >= 15:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = winsorize(all_etf_combined_df.loc[date].loc[market]['Factor'], limits=1/n_etf)
            elif n_etf > 0:
                all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = all_etf_combined_df.loc[date].loc[market]['Factor']
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'] = zscore(all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'])
    all_etf_combined_df = all_etf_combined_df.rename({'Normalized Winsorized Factor': factor_to_analyze}, axis=1)
    if all_etf_all_factor_df is None:
        all_etf_all_factor_df = all_etf_combined_df[['f_rtn', factor_to_analyze]]
    else:
        all_etf_all_factor_df = all_etf_all_factor_df.join(all_etf_combined_df[[factor_to_analyze]], how='outer')

Put/Call Ratio
Short Interest/Market Capitalization


In [19]:
all_etf_all_factor_df.reset_index()

,Date,Market,Ticker,f_rtn,pc_ratio,short_int_ratio
0,2016-01-04,Developed,EWA,-0.071979,0.899203,1.801277
1,2016-01-04,Developed,EWC,-0.013368,0.888940,1.610433
2,2016-01-04,Developed,EWD,-0.060830,-0.388471,-0.342597
3,2016-01-04,Developed,EWG,-0.055149,-0.525739,-0.731667
4,2016-01-04,Developed,EWH,-0.098062,-0.171724,1.033072
...,...,...,...,...,...,...
32931,2019-12-30,Emerging,GREK,-0.038922,-0.902005,0.731432
32932,2019-12-30,Emerging,KSA,-0.021221,-0.809799,1.902926
32933,2019-12-30,Emerging,MCHI,-0.042813,-0.039688,0.013867
32934,2019-12-30,Emerging,THD,-0.082510,0.023937,-1.068088


In [20]:
all_etf_all_factor_df.reset_index().to_csv('modeling data/economic.csv')